In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上的E2E ML：MLOps阶段6：开始使用Vertex AI批量预测和可解释的AI进行AutoML表格模型

在Colab中运行
在GitHub上查看
在Vertex AI Workbench中打开

## 概述

本教程演示了如何使用Vertex AI SDK创建表格二元分类模型，并使用[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行可解释的AI批量预测。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML表格二元分类模型，然后使用Vertex AI SDK进行批量预测，同时使用可解释AI。您还可以选择使用`gcloud`命令行工具或在线使用Cloud Console来创建和部署模型。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Datasets`
- `Vertex AI Models`
- `BigQuery`

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练一个`AutoML`表格模型。
- 使用CSV输入进行批量预测。
- 使用JSONL对象输入进行批量预测。
- 使用JSONL列表输入进行批量预测。
- 使用BigQuery表输入进行批量预测。
- 使用解释进行批量预测。

使用批量预测和使用在线预测之间有一个关键区别：

* 预测服务：对整个实例集合（即一个或多个数据项）进行即时预测并实时返回结果。

* 批量预测服务：对整个实例集合进行排队（批处理）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

数据集

这篇教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)。这个数据集不需要任何特征工程。你在这篇教程中使用的数据集版本存储在一个公共云存储桶中。

### 费用

本教程使用Google Cloud的计费组件：

- Vertex AI
- Cloud Storage
- BigQuery

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage定价](https://cloud.google.com/storage/pricing)和[BigQuery定价](https://cloud.google.com/bigquery/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，则您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在Python 3虚拟环境中运行的Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简明的指令：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，可以在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，可以在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开这个笔记本。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-bigquery
! pip3 install tensorflow $USER_FLAG -q

重新启动内核

安装了额外的软件包后，需要重新启动笔记本内核，以便它能找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在您开始之前

GPU运行时

本教程不需要GPU运行时。

设置您的Google Cloud项目

**无论您使用的是笔记本环境，以下步骤都是必需的。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs, Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter会将以`!`为前缀的行作为shell命令运行，并会替换以`$`为前缀的Python变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 认证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过了身份验证。

**如果您正在使用Colab**，运行下方的单元格，并按提示进行操作通过oAuth进行账户认证。

**否则**，请按照以下步骤进行：

在Cloud控制台上，进入[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入名称，然后点击**创建**。

在**授予该服务账户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件会下载到您的本地环境中。

将您的服务账户密钥路径输入到下方的单元格作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境是什么，都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关的所有数据在会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格以创建您的Cloud Storage存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查 Cloud Storage 存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 创建 BigQuery 客户端

在本教程中，您将使用与预训练模型训练时使用的相同公共 BigQuery 表中的数据。您将创建一个客户端接口，然后用它来访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

现在您准备好开始创建自己的AutoML表格二分类模型。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的 Bank Marketing 数据集的一个版本，使用 CSV 索引文件。

首先快速查看数据。您可以通过统计 CSV 索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

您还需要知道训练目的，标签列的标题名称，这个名称保存为`label_column`。对于这个数据集，它是 CSV 文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，需要传入以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源中。

该操作可能需要几分钟的时间。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="Bank Marketing" + "_" + UUID, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建和运行训练流程

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流程，2）运行该流程。

#### 创建训练流程

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练流程，具有以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类别分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是用于训练流程的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="bank_" + UUID,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

#### 运行训练管道

接下来，通过调用`run`方法并传入以下参数来运行DAG以启动训练作业：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则服务可能会在达到整个预算之前完成训练，如果认为无法进一步改善模型目标测量。

完成`run`方法后会返回`Model`资源。

训练管道的执行可能需要长达 8 小时。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="bank_" + UUID,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 批量预测介绍

批量预测提供了对大量预测请求进行离线批处理的能力。资源只在批处理过程中被分配，然后在批请求完成时取消分配。结果存储在云存储中，与在线预测不同，在在线预测中，结果将作为HTTP响应数据包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首要的是，您的模型服务器中的Web服务器必须支持JSONL格式，该Web服务器将其转换为模型输入接口或服务函数接口直接支持的格式。对于批处理预测，该JSONL格式被称为“枢纽”格式。

### 批量预测作业的输入格式

批处理服务器接受以下输入格式的AutoML表格模型：

- JSONL
- CSV
- BigQuery 表

### 批量预测作业的输出格式

批处理服务器接受以下AutoML表格模型的输出格式：

- JSONL
- CSV
- BigQuery 表

### 枢纽格式

批处理服务器将输入格式转换为“枢纽”（JSONL）格式，如下所示：

**JSONL**

每个输入行（请求）应包含一个且仅一个有效的json值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器使用相同格式生成枢纽数据。生成的枢纽数据随后被包装到一个载荷请求中：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的CSV标题总是被忽略。字符串字段需要显式地用双引号括起来，否则该行将被丢弃，并且会输出解析错误信息到错误文件。非引号的值总是被传输为浮点数。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每行都转换为JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器使用相同格式生成枢纽数据。生成的枢纽数据随后被包装到一个载荷请求中：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例由apache_beam.io.tfrecordio模块以二进制形式读取。然后将二进制对象序列化为ASCII字符串。预测服务器负责知道解码器以还原实例。

    {"instances": [
     {"b64","b64编码的ASCII字符串"},
     {"b64","b64编码的ASCII字符串"}
    ]}

**文件列表**

FileList格式包含文件列表。在“FileList”文件中，每一行指定一个文件路径，指定为云存储位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器将文件读取为二进制文件。二进制对象被序列化为ASCII字符串。

    {"instances": [
     {"b64","b64编码的ASCII字符串"},
     {"b64","b64编码的ASCII字符串"}
    ]}

### 使用CSV输入制作批量输入文件

现在创建一个批量输入文件，并将其存储在本地的云存储桶中。在这个例子中，您可以使用CSV格式来进行输入和输出。对于输入文件，CSV文件的布局如下：

- 第一行是带有特征（字段）名称的标题。
- 每个剩余的行都是一个独立的预测请求，具有相应的特征值。

例如：

    "feature_1", "feature_2". ...
    value_1, value_2, ...

制作测试项目

您可以使用合成数据作为测试数据项目。

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = BUCKET_URI + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

### 发出批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict()方法来进行批量预测，以下是参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `sync`：如果设置为True，则调用在等待异步批处理作业完成时会阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="csv",
    sync=False,
)

print(batch_predict_job)

等待批量预测任务完成

接下来，等待批量任务完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测任务完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出存储桶。您可以调用iter_outputs()方法获取生成的结果的每个云存储文件的列表。每个文件都以CSV格式包含一个或多个预测请求：

- CSV标题 + 预测标签
- CSV行 + 预测，每个预测请求一个

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

#### 删除批量预测作业

您可以使用`delete（）`方法删除您的`Vertex AI批量预测`作业。

In [ ]:
batch_predict_job.delete()

使用JSONL输入制作批量输入文件

现在制作一个批量输入文件，将其存储在您的本地云存储桶中。在这个例子中，您可以使用JSONL格式作为输入和输出。对于输入，JSONL文件的布局如下：

{"feature1": "value1", "feature2": "value2", ..., "featureN": "valueN"}

制作测试项目

您使用合成数据作为测试数据项目。

In [ ]:
import json

INSTANCE_1 = {
    "Age": "58",
    "Job": "management",
    "MaritalStatus": "married",
    "Education": "tertiary",
    "Default": "no",
    "Balance": "2143",
    "Housing": "yes",
    "Loan": "no",
    "Contact": "unknown",
    "Day": "5",
    "Month": "may",
    "Duration": "261",
    "Campaign": "1",
    "PDays": "-1",
    "Previous": "0",
    "POutcome": "unknown",
}

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    f.write(json.dumps(INSTANCE_1) + "\n")

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用 batch_predict() 方法进行批量预测，使用以下参数：

- 'job_display_name'：批量预测作业的可读名称。
- 'gcs_source'：一个或多个批量请求输入文件的列表。
- 'gcs_destination_prefix'：用于存储批量预测结果的云存储位置。
- 'instances_format'：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- 'predictions_format'：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- 'sync'：如果设置为True，则调用将在等待异步批量作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的Cloud Storage输出存储桶。您可以调用iter_outputs()方法获取每个用于存储结果的Cloud Storage文件的列表。每个文件以JSONL格式包含一个或多个预测请求：

    {"instance": {"feature1": "value1", ..., "featureN": "valueN"}

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

#### 删除批量预测作业

您可以使用`delete（）`方法删除您的`Vertex AI Batch Prediction`作业。

In [ ]:
batch_predict_job.delete()

### 发起批处理解释请求

现在您的模型资源已经训练完成，您可以通过调用`batch_predict()`方法进行批处理预测，带有以下参数：

- `job_display_name`：批处理预测作业的可读名称。
- `gcs_source`：一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批处理预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以选择'csv'或'jsonl'。默认值为'jsonl'。
- `predictions_format`：输出预测结果的格式，可以选择'csv'或'jsonl'。默认值为'jsonl'。
- `generate_explanations`：设置为`True`以生成解释。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

对于解释部分，仅支持'jsonl'和'bigquery'输出格式。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    generate_explanation=True,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测任务完成

接下来，等待批量任务完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测任务完成。

In [ ]:
batch_predict_job.wait()

### 获取解释

接下来，从完成的批量预测作业中获取解释结果。

结果被写入到您在批量预测请求中指定的Cloud Storage输出桶中。您调用iter_outputs()方法来获取包含结果的每个Cloud Storage文件的列表。每个文件以JSONL格式包含一个或多个解释请求：

- instance: 用于预测的特征值。
- prediction: 预测值。
- explanation: 每个特征值的归因。

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation"):
        explanation_results.append(blob.name)

tags = list()
for explanation_result in explanation_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

使用JSONL输入（列表）创建批处理输入文件

或者，您可以将JSONL格式指定为列表（与对象相对）。在这种情况下，值必须按照为训练模型指定的相同列表顺序排列。

JSONL文件的布局如下：

["值1", "值2", ..., "值N"]

制作测试项

您可以使用合成数据作为测试数据项。

In [ ]:
INSTANCE_1 = [
    58,
    "management",
    "married",
    "tertiary",
    "no",
    2143,
    "yes",
    "no",
    "unknown",
    5,
    "may",
    261,
    1,
    -1,
    0,
    "unknown",
]

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    f.write(json.dumps(INSTANCE_1) + "\n")

### 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用 batch_predict() 方法来进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    sync=False,
)

print(batch_predict_job)

### 等待批处理预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测任务中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出存储桶中。您可以调用 iter_outputs() 方法获得包含结果的每个云存储文件的列表。每个文件以 JSONL 格式包含一个或多个预测请求：

    {"instance": {"feature1": "value1", ..., "featureN": "valueN"}

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

#### 删除批量预测作业

您可以使用`delete()`方法删除`Vertex AI Batch Prediction`作业。

In [ ]:
batch_predict_job.delete()

使用BigQuery输入格式执行批量预测

接下来，您将执行相同的批处理作业，只不过输入格式为BigQuery表。当输入格式为BigQuery表时，输出格式也必须为BigQuery表。

### 从CSV文件创建一个BigQuery数据集

您可以使用BigQuery的`create_dataset()`和`load_table_from_uri()`方法从CSV文件创建一个BigQuery数据集，步骤如下：

- `create_dataset()`: 创建一个空的BigQuery数据集，具有以下参数：
  - `dataset_ref`: 从数据集ID创建的`DatasetReference`，例如samples。
- `load_table_from_uri()`: 将一个或多个CSV文件加载到相应数据集中的表中，具有以下参数：
  - `url`: 存储在云存储中的一个或多个CVS文件的集合。
  - `table`: 表的`TableReference`。
  - `job_config`: 加载CSV数据的规格。

了解更多关于[将CSV数据导入到BigQuery](https://www.tensorflow.org/io/tutorials/bigquery#import_census_data_into_bigquery)的信息。

In [ ]:
# Create test data w/o the label
output = ! gsutil cat $IMPORT_FILE | head -n 10
rows = []
for i in range(0, 10):
    rows.append(output[i][:-2])

with tf.io.gfile.GFile(BUCKET_URI + "/test.csv", "w") as f:
    for row in rows:
        f.write(row + "\n")

LOCATION = "us"

CSV_SCHEMA = [
    bigquery.SchemaField("Age", "INTEGER"),
    bigquery.SchemaField("Job", "STRING"),
    bigquery.SchemaField("MaritalStatus", "STRING"),
    bigquery.SchemaField("Education", "STRING"),
    bigquery.SchemaField("Default", "STRING"),
    bigquery.SchemaField("Balance", "INTEGER"),
    bigquery.SchemaField("Housing", "STRING"),
    bigquery.SchemaField("Loan", "STRING"),
    bigquery.SchemaField("Contact", "STRING"),
    bigquery.SchemaField("Day", "INTEGER"),
    bigquery.SchemaField("Month", "STRING"),
    bigquery.SchemaField("Duration", "INTEGER"),
    bigquery.SchemaField("Campaign", "INTEGER"),
    bigquery.SchemaField("PDays", "INTEGER"),
    bigquery.SchemaField("Previous", "INTEGER"),
    bigquery.SchemaField("POutcome", "STRING"),
]

DATASET_ID = "batch"
TABLE_ID = "slice1"


def create_bigquery_dataset(dataset_id):
    dataset = bigquery.Dataset(
        bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id)
    )
    dataset.location = "us"

    try:
        dataset = bqclient.create_dataset(dataset)  # API request
        return True
    except Exception as err:
        print(err)
        if err.code != 409:  # http_client.CONFLICT
            raise
    return False


def load_data_into_bigquery(url, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.schema = CSV_SCHEMA
    job_config.skip_leading_rows = 1  # heading

    load_job = bqclient.load_table_from_uri(url, table, job_config=job_config)
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = bqclient.get_table(table)
    print("Loaded {} rows.".format(destination_table.num_rows))

    return destination_table


bq_table = load_data_into_bigquery(BUCKET_URI + "/test.csv", DATASET_ID, TABLE_ID)
print(bq_table)

发送批量预测请求

再次，使用`batch_predict()`方法发送批量预测请求，但是参数有以下更改：

- `instances_format`：设置为'bigquery'
- `predictions_format`：设置为'bigquery'
- 使用`bigquery_source`代替`gcs_source`。
- 使用`bigquery_destination_prefix`代替`gcs_destination_prefix`。

In [ ]:
# The name of the job
BATCH_PREDICTION_JOB_NAME = "bank-" + UUID

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results_bq"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

BQ_TABLE_SOURCE = f"bq://{PROJECT_ID}.{bq_table.dataset_id}.{bq_table.table_id}"

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="bigquery",
    predictions_format="bigquery",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    bigquery_source=BQ_TABLE_SOURCE,
    bigquery_destination_prefix=f"bq://{PROJECT_ID}.batch",
    model_parameters=None,
    sync=False,
)

### 等待批量预测任务完成

接下来，等待批量任务完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测任务完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批处理预测作业中获取结果。

结果将写入到一个 BigQuery 表中，该表位于您指定的 BigQuery 数据集路径下作为目的地。批处理服务器创建表，表位置由以下参数指定：

`batch_prediction_job.output_info.bigquery_output_dataset`：项目和数据集组件。
`batch_prediction_job.output_info.bigquery_output_table`：表组件。

In [ ]:
BQ_RESULTS_TABLE = (
    batch_prediction_job.output_info.bigquery_output_dataset
    + "."
    + batch_prediction_job.output_info.bigquery_output_table
)

print(BQ_RESULTS_TABLE)

table = bigquery.TableReference.from_string(BQ_RESULTS_TABLE[5:])

rows = bqclient.list_rows(table, max_results=10)

for row in rows:
    print(row)
    for key, value in row.items():
        pass

您可以使用`delete()`方法删除您的`Vertex AI批量预测`作业。

In [ ]:
batch_predict_job.delete()

删除模型

接下来，您可以使用`delete()`方法删除`顶点AI模型`资源。

In [ ]:
model.delete()

删除数据集

接下来，您可以使用`delete()`方法删除`Vertex AI数据集`资源。

In [ ]:
dataset.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}